In [37]:
import pandas as pd
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from google.cloud import translate_v2 as translate
from tqdm import tqdm
import string
import preprocessor as p

In [38]:
file_name = input('file name : ')

file name : 2022-05-16_counterstrike


In [39]:
# 원본 파일 읽어옴
df = pd.read_csv('./corpus/reddit/' + file_name + '.csv', index_col=0, encoding="UTF-8").dropna()
df.drop_duplicates(subset=['Text'], inplace=True)

In [40]:
df

,Datetime,Text
0,2022-05-16,{''}
3,2022-05-16,"{""I noticed there weren't many pug servers up, and I was to lazy to look for them. So, I formulated an idea which could possibly work, but I'm not sure if it would get any attention. It would have Skill Based Matchmaking, and basically anything you would expect from a matchmaking program. I just want to hear people's opinions, and see if this is something people would enjoy :)""}"
14,2022-05-14,{'[https://www.youtube.com/watch?v=RpGPAy60mkY&t=1s](https://www.youtube.com/watch?v=RpGPAy60mkY&t=1s)'}
15,2022-05-14,{'My specs are good but when I use any settings it fluctuates to around 60 to 40. Any fixes?'}
16,2022-05-14,"{'Recently, my Local Game Store is selling a PC Retail copy of Counter-Strike Condition Zero on PC CD-ROM and I am very interested. It is similar to the Photo Shown Below. What I need to know is, will this copy have have the Single Player Campaign Included on Disc? I don\'t know much about Counter-Strike, I\'ve only played Global Offensive. But I am a Video Game Collector and this interests me a lot. I\'m not interested in Multiplayer of Condition Zero.\n\nThese are my questions and concerns:\n\nWill purchasing this version of Condition Zero include the Main Campaign? \nWhat Patches do I need to download and Install?\n\nIs the Campaign worth it?\n\nWhat is ""Deleted Scenes""? Is it possible to get it online?\n\nPlease let me know in full detail. Thank you.\n\n&#x200B;\n\nhttps://preview.redd.it/9g5w9nte2cz81.png?width=1600&format=png&auto=webp&s=11e9b5d033ad8018b5d0cbd5799617df08d38078'}"
18,2022-05-14,{'I cheated and got my main banned a long time ago and want to get back into cs but a fresh acc finds no games.'}
24,2022-05-13,"{""I bought prime around 1 year ago in april 2021. I stopped playing since then and now that I'm back on the game my prime status disappeared. Im lvl 17 btw, how can I reactivated it? I swear i bought it, i'm sure 200%\nEdit: when i click on my profile picture in the top right, theres an icon that says “prime enabled account”. But when i try to queue up there are none prime queues, and in ALL my ranked games i had teamkillers and leavers, whats wrong?\nIs the “official matchmaking” the prime queue?""}"
25,2022-05-13,{'Cs'}
27,2022-05-13,"{"" \n\nI'm now touching the market for the first time in years and i was looking at the gold Sickers from 2020 RMR and noticed a gigantic drop in all stickers around 5-7 april 2021. Can someone explain me this drop please?\n\nAnd if possibl give me advice on these 2020 rmr stickers if they are a good investment rn or any other stickers i could possibly invest in?\n\nthank you""}"
31,2022-05-13,{'Is counter strike warzone a portable version of counter strike source. And will it work without administrator privellages on a computer?'}


In [26]:
# 전처리 함수 내 사용
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "&gt;" : "is greater than", "&lt;" : "is less than", "&nbsp;": " ", "&amp;" : "and", "&quot;" : '"'}

In [27]:
def preprocessing(data) :
    data = re.sub('’|‘|´|`', '\'', data)
    data = re.sub('\\n', ' ', data)
    # normalize abbreviation
    data = ' '.join([contractions[t] if t in contractions else t for t in data.split(" ")])
    # clean corpus to remove emoji and URL
    data = p.clean(data)
    # remove numbers and punctuations
    data = re.sub(r'[^A-Za-z ]', ' ', data)
    # lowering
    data = data.lower()
    # remove duplicated alphabets
    data = re.sub(r'([A-Za-z])\1{2,}', r'\1', data)
    
    data = re.sub('league of legends', 'leagueoflegends', data)
    
    return data

In [28]:
df['Text'] = df['Text'].apply(lambda x : preprocessing(x))

In [29]:
df_temp = df[df['Text'] == ''].index
df = df.drop(df_temp)
result = df[['Datetime', 'Text']]
result['Text'] = result['Text'].apply(lambda x : ' '.join(x.split()))

In [30]:
stop_words = set(stopwords.words('english')) 
result['Clean'] = result['Text'].apply(lambda x : ' '.join(word for word in x.split() if not word in stop_words if len(word) > 1))
result

,Datetime,Text,Clean
1,2022-05-16,tought i would post this here,tought would post
2,2022-05-16,the unranked is goldnova max love this matchmaking,unranked goldnova max love matchmaking
3,2022-05-16,had valve gotten more uncultured,valve gotten uncultured
4,2022-05-16,i noticed there were not many pug servers up and i was to lazy to look for them so i formulated an idea which could possibly work but i am not sure if it would get any attention it would have skill based matchmaking and basically anything you would expect from a matchmaking program i just want to hear people s opinions and see if this is something people would enjoy,noticed many pug servers lazy look formulated idea could possibly work sure would get attention would skill based matchmaking basically anything would expect matchmaking program want hear people opinions see something people would enjoy
5,2022-05-16,i bought prime around year ago in april i stopped playing since then and now that i am back on the game my prime status disappeared im lvl btw how can i reactivated it i swear i bought it i am sure edit when i click on my profile picture in the top right theres an icon that says prime enabled account but when i try to queue up there are none prime queues and in all my ranked games i had teamkillers and leavers whats wrong is the official matchmaking the prime queue,bought prime around year ago april stopped playing since back game prime status disappeared im lvl btw reactivated swear bought sure edit click profile picture top right theres icon says prime enabled account try queue none prime queues ranked games teamkillers leavers whats wrong official matchmaking prime queue
6,2022-05-16,i am now touching the market for the first time in years and i was looking at the gold sickers from rmr and noticed a gigantic drop in all stickers around april can someone explain me this drop please and if possibl give me advice on these rmr stickers if they are a good investment rn or any other stickers i could possibly invest in thank you,touching market first time years looking gold sickers rmr noticed gigantic drop stickers around april someone explain drop please possibl give advice rmr stickers good investment rn stickers could possibly invest thank
7,2022-05-16,hello ive recently been very interested in making cs skins and make quite a lot i think its a waste to not share them on the internet but im not sure wheres the best place to do so are there any communities for this kind of stuff thanks,hello ive recently interested making cs skins make quite lot think waste share internet im sure wheres best place communities kind stuff thanks
8,2022-05-16,can someone please put up a stattrak stiletto knife rust coat for it would be really funny and awesome and cool and sick and ill kiss thanks big fellas,someone please put stattrak stiletto knife rust coat would really funny awesome cool sick ill kiss thanks big fellas
9,2022-05-16,hey fellas so i am facing this issues since a long time i have added the b bind to dropping the bomb directly when pressing it to save time and have saved it in my auto exec and for shift bind to lower my volume and increase the surrounding sounds however whenever i open the game and execute these auto exec commands the game automatically deletes the buy menu option b from the settings menu and i no longer can buy guns using b and also removes shift for walking and i can no longer use it to walk can anyone help me to bind b to both buying guns and dropping the bomb at my convenience is there a way to trigger b for buying whenever i want to buy guns and also trigger it to drop bomb whenever i want to use it to drop bomb and also the exact same thing for the shift key please help,hey fellas facing issues since long time added bind dropping bomb directly pressing save time saved auto exec shift bind lower volume increase surrounding sounds however whenever open game execute auto exec commands game automatically deletes buy menu option settings menu 

In [31]:
result_temp = result[result['Clean'] == ''].index
print(result_temp)
result = result.drop(result_temp).set_index("Datetime")

Int64Index([11, 18], dtype='int64')


In [32]:
result.to_csv('sample_cs_preprocessed.csv')